## Notes on GIS

Based on spherical earth model, we can use a spherical approximation for most local calculations.
<br>The conversion between meters and degrees works as follows:

Latitude Offset<br>
The distance of $1^\circ$ of latitude is approximately constant everywhere on Earth:$$1^\circ \text{ Latitude} \approx 111,320 \text{ meters}$$Therefore,
the latitude offset ($\Delta lat$) is:$$\Delta lat = \frac{\text{distance in meters}}{111,320}$$



Longitude Offset<br>
Lines of longitude converge at the poles.<br>
To find the distance of $1^\circ$ of longitude, one must multiply the **equatorial constant** by the cosine of the latitude:
$$1^\circ \text{ Longitude} \approx 111,320 \times \cos(lat)$$
Thus, the longitude offset
($\Delta lon$) is:$$\Delta lon = \frac{\text{distance in meters}}{111,320 \times \cos(lat \times \frac{\pi}{180})}$$

Calculation Steps<br>
Given a point $(lat, lon)$ and a distance $d$ in meters:

Calculate the Delta Latitude:
 - $\Delta lat = d / 111,320$

Calculate the Delta Longitude:
 - $\Delta lon = d / (111,320 \times \cos(lat \times \frac{\pi}{180}))$

Define the Bounding Box Corners:
 - Min Lat: $lat - \Delta lat$
 - Max Lat: $lat + \Delta lat$
 - Min Lon: $lon - \Delta lon$
 - Max Lon: $lon + \Delta lon$

In [ ]:
import math

def get_bounding_box(lat, lon, distance_m):
    # Earth's radius constants
    lat_const = 111320.0

    # Calculate offsets
    delta_lat = distance_m / lat_const
    delta_lon = distance_m / (lat_const * math.cos(math.radians(lat)))

    return {
        "min_lat": lat - delta_lat,
        "max_lat": lat + delta_lat,
        "min_lon": lon - delta_lon,
        "max_lon": lon + delta_lon
    }

Important ConsiderationsThe Poles:
- As location approach $90^\circ$ or $-90^\circ$ latitude, $\cos(lat)$ approaches zero, making $\Delta lon$ extremely large.
- If a bounding box crosses a pole, the above math for a simple rectangle fails.

Antimeridian:
- If longitude is near $180^\circ$ or $-180^\circ$, bounding box may "wrap around" the world.
- It should check if min_lon < -180 or max_lon > 180 and adjust accordingly.

In [ ]:
import math

def normalize_coordinates(lat, lon):
    """
    Wraps longitude to [-180, 180] and latitude to [-90, 90].
    If latitude wraps over a pole, the longitude is flipped by 180 degrees.
    """

    # 1. Wrap Longitude first to a general 360 range
    # Formula: ((x + 180) % 360) - 180
    lon = ((lon + 180) % 360) - 180

    # 2. Wrap Latitude
    # Latitude is more complex because it reflects back from the poles
    # We use a double-modulo approach to simulate walking over the poles
    lat = (lat + 180) % 360
    if lat < 0:
        lat += 360
    lat -= 180

    if lat > 90:
        lat = 180 - lat
        lon += 180
    elif lat < -90:
        lat = -180 - lat
        lon += 180

    # 3. Final Longitude Wrap (in case the latitude flip pushed lon out of bounds)
    lon = ((lon + 180) % 360) - 180

    return round(lat, 6), round(lon, 6)

# --- Test Cases ---
print(f"Standard Point: {normalize_coordinates(40, -74)}")        # (40.0, -74.0)
print(f"Past Antimeridian: {normalize_coordinates(10, 190)}")     # (10.0, -170.0)
print(f"Past North Pole: {normalize_coordinates(100, 0)}")        # (80.0, -180.0)
print(f"Multiple Revolutions: {normalize_coordinates(10, 730)}")  # (10.0, 10.0)

In [ ]:
query="""
SELECT ST_Envelope(
    ST_Buffer(
        ST_SetSRID(ST_MakePoint(lon, lat), 4326)::geography,
        1000 -- distance in meters
    )::geometry
) AS mbb;
"""

In [ ]:
query="""
SELECT ST_Transform(
    ST_Envelope(
        ST_Buffer(
            ST_Transform(ST_SetSRID(ST_MakePoint(lon, lat), 4326), 3857),
            1000
        )
    ),
    4326
) AS mbb;
"""

The Earth's rotation creates centrifugal force that pushes mass outward at the equator. This results in an equatorial bulge, making the planet approximately 21.4 km wider at the center than it is from pole to pole. Because the Earth is not a perfect sphere but an oblate spheroid (bulging at the equator and flattened at the poles), there is no single "radius."


| Type | Value in Meters | Description |
| -----|--------|------|
| Equatorial Radius ($a$) | 6,378,137.0 m | Distance from the center to the Equator. |
| Polar Radius ($b$) | 6,356,752.3 m | Distance from the center to the North/South Poles. |
| Mean Radius ($R_1$) | 6,371,008.8 m | The average of the three semi-axes ($\frac{2a + b}{3}$). |
| Volumetric Radius | 6,371,000.8 m | Radius of a sphere with the same volume as Earth.|

- **For General Calculations**: Use the IUGG mean radius of 6,371,000 meters. This is the most common value used in simple spherical distance formulas (like the Haversine formula).
- **For High-Precision GIS**: Use the WGS84 Equatorial radius (6,378,137 m) as  "Semi-major axis" ($a$).
- **For Bounding Box**: Most developers use 6,371,000 m or 6,378,137 m for the constant in the $\Delta lat$ and $\Delta lon$ math  discussed earlier.